In [3]:
pip install selenium pandas


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Set up Selenium options
options = Options()
options.add_argument("--headless")  # Run in headless mode (remove this for debugging)
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# List to store scraped data
smartphones = []

# Loop through multiple pages (1 to 50)
for page in range(1, 51):
    URL = f"https://www.jumia.com.ng/catalog/?q=smart+phones&page={page}"
    driver.get(URL)
    
    # Wait for products to load
    try:
        products = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.prd')))
    except:
        print(f"No products found on page {page}. Ending scrape.")
        break
    
    for product in products:
        try:
            name = product.find_element(By.CSS_SELECTOR, "h3.name").text
            link = product.find_element(By.CSS_SELECTOR, "a.core").get_attribute("href")
            price = product.find_element(By.CSS_SELECTOR, "div.prc").text
            discount = product.find_element(By.CSS_SELECTOR, "div.bdg._dsct._sm").text if product.find_elements(By.CSS_SELECTOR, "div.bdg._dsct._sm") else "No Discount"
            
            # Open product link in new tab for extra details
            driver.execute_script("window.open('');")  # Open new tab
            driver.switch_to.window(driver.window_handles[1])  # Switch to new tab
            driver.get(link)
            time.sleep(3)  # Allow time for page to load
            
            # Extract additional details
            try:
                reviews = driver.find_element(By.CSS_SELECTOR, "div.-pvs.-mvxs").text
            except:
                reviews = "No Reviews"
                
            try:
                seller = driver.find_element(By.CSS_SELECTOR, "div.-fs14.-pvxs").text
            except:
                seller = "No Seller Info"

            try:
                warranty = driver.find_element(By.XPATH, "//div[contains(text(), 'Warranty')]/following-sibling::div").text
            except:
                warranty = "No Warranty Info"

            try:
                return_policy = driver.find_element(By.XPATH, "//div[contains(text(), 'Return')]/following-sibling::div").text
            except:
                return_policy = "No Return Policy"

            # Store data
            smartphones.append({
                "Name": name,
                "Price": price,
                "Discount": discount,
                "Reviews": reviews,
                "Seller": seller,
                "Warranty": warranty,
                "Return Policy": return_policy,
                "Link": link
            })

            print(f"Scraped: {name}")

            # Close new tab and return to main page
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print("Error extracting product:", e)
    
    print(f"✅ Page {page} scraped successfully.")

# Close browser
driver.quit()

# Save data to CSV
df = pd.DataFrame(smartphones)
df.to_csv("jumia_smartphones.csv", index=False, encoding="utf-8")

print(f"\n✅ Total Smartphones Scraped: {len(smartphones)}")
print("📁Data saved as 'jumia_smartphones.csv'.")


Scraped: Samsung Galaxy A05 6.7" 4GB RAM/64GB ROM Android 13 - Light Green
Scraped: itel A50 6.6" 2GB RAM/64GB ROM Android 12 - Black
Scraped: itel RS4 256GB ROM+24(12+12) GB RAM Lurex Black
Scraped: Infinix Smart 9HD 6.7" 4GB RAM/64GB ROM Black
Scraped: Tecno Spark 30C 6.6" 4GB RAM/128GB ROM Android 14 - Black
Scraped: Infinix Smart 9HD 6.7" 4GB RAM/64GB ROM Titanium
Scraped: XIAOMI Redmi 14C 6.88'' 4GB RAM/128GB ROM Android 14 -Midnight Black
Scraped: Tecno Pop 9 6.67" 3GB RAM /64GB ROM Android 14 - Black
Scraped: AGM Note N1 6.52" 8GB RAM / 128GB Expandable ROM Android 13 - Grey
Scraped: itel S25 Ultra 6.78" AMOLED 256+8 4G Android-Ocean
Scraped: Samsung Galaxy A05 6.7" 4GB RAM/64GB ROM Android 13 - Black..
Scraped: Tecno Pop 9 6.67" 4GB RAM / 128GB ROM Android 14 - Black
Scraped: itel P55+ 6.6″ HD 8GB RAM/128GB ROM Android 12 - Green
Scraped: itel A06 6.6" HD+, 32GB ROM + Up To 4GB RAM, 4000mAh, 4G LTE, 8MP
Scraped: Infinix Hot 50 Pro+ 6.78" 8GB RAM/128GB ROM Android 14- Black
Scra

In [14]:
import pandas as pd

# Convert list to DataFrame
df = pd.DataFrame(smartphones)

# Save as CSV
df.to_csv("jumia_smartphones.csv", index=False, encoding="utf-8")

print("Data saved successfully!")

Data saved successfully!


In [17]:
print(len(smartphones))

2000


In [18]:
df=pd.read_csv('jumia_smartphones.csv')
df.head()

,Name,Price,Discount,Reviews,Seller,Warranty,Return Policy,Link
0,"Samsung Galaxy A05 6.7"" 4GB RAM/64GB ROM Andro...","₦ 123,302",No Discount,No Reviews,No Seller Info,No Warranty Info,No Return Policy,https://www.jumia.com.ng/samsung-galaxy-a05-6....
1,"itel A50 6.6"" 2GB RAM/64GB ROM Android 12 - Black","₦ 91,800",17%,No Reviews,No Seller Info,No Warranty Info,No Return Policy,https://www.jumia.com.ng/itel-a50-6.6-2gb-ram6...
2,itel RS4 256GB ROM+24(12+12) GB RAM Lurex Black,"₦ 199,000",20%,No Reviews,No Seller Info,No Warranty Info,No Return Policy,https://www.jumia.com.ng/itel-rs4-256gb-rom241...
3,"Infinix Smart 9HD 6.7"" 4GB RAM/64GB ROM Black","₦ 110,658",18%,No Reviews,No Seller Info,No Warranty Info,No Return Policy,https://www.jumia.com.ng/infinix-smart-9hd-6.7...
4,"Tecno Spark 30C 6.6"" 4GB RAM/128GB ROM Android...","₦ 135,915",14%,No Reviews,No Seller Info,No Warranty Info,No Return Policy,https://www.jumia.com.ng/tecno-spark-30c-6.6-4...


In [20]:
df.to_csv(r"C:\Users\USER\Documents\Project\jumia_smartphones.csv", index=False, encoding="utf-8")